In [1]:
!pip install geopy

In [53]:
import rasterio
from rasterio.transform import Affine
from rasterio.mask import mask
from shapely.geometry import Point, mapping
from pyproj import Proj, transform
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from geopy.geocoders import Nominatim
import dask.array as da
import dask.delayed
import pandas as pd
from pyproj import Transformer
from dask.delayed import delayed
from rasterio.windows import Window
from dask import delayed
import os
import xarray as xr
import math
import pandas as pd
import scipy.stats as stats
import matplotlib as plt

In [3]:
file_openwater_area = "local-data/1_openwater_area.csv"
polygons_with_geolocation = pd.read_csv("local-data/Hylakes_with_location.csv")
openwater = pd.read_csv(file_openwater_area)

/tmp/ipykernel_8536/3755981705.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  polygons_with_geolocation = pd.read_csv("local-data/Hylakes_with_location.csv")


In [4]:
lake_name = 'Great Salt'

# Assuming the dataset has a 'name' column with lake names
specific_lake = polygons_with_geolocation[polygons_with_geolocation['Lake_name'] == lake_name]

if not specific_lake.empty:
    # Get the first matching lake's coordinates
    lake_geometry = specific_lake.iloc[0]
    lat = specific_lake['Pour_lat'].iloc[0]
    lon = specific_lake['Pour_long'].iloc[0]
    print(lat, lon)
else:
    print("Lake not found")


41.410417 -112.83125


In [5]:
rt_path = "/home/derick/workspace/IMSACODE/IMSA2023LakeData/land_used_dataset/Winkler_etal_2021_HILDA_allfiles/hildap_vGLOB-1.0-f_netcdf/"
urbanization_files = [rt_path + item for item in os.listdir(rt_path) if item.endswith(".nc")]

In [6]:
data_urbanization = xr.open_mfdataset(urbanization_files, combine='by_coords')

/home/derick/workspace/IMSACODE/IMSA2023LakeData/virtualenv/lib/python3.10/site-packages/dask/array/core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 34
  result = blockwise(
/home/derick/workspace/IMSACODE/IMSA2023LakeData/virtualenv/lib/python3.10/site-packages/dask/array/core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 34
  result = blockwise(


In [44]:
lat_idx = (np.abs(data_urbanization['latitude'] - 41.334761)).argmin()
lon_idx = (np.abs(data_urbanization['longitude'] - -113.946334)).argmin()
LULC_states = data_urbanization['LULC_states'][:,lat_idx, lon_idx].values.astype(float).tolist()

In [8]:
LULC_states 

[66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0,
 66.0]

In [9]:
import geopandas as gpd
import rasterio
from rasterio.plot import show


In [10]:
import xarray as xr
dataset = xr.open_dataset('land_used_dataset/Winkler_etal_2021_HILDA_allfiles/hildap_vGLOB-1.0-f_netcdf/hildaplus_GLOB-1-0-f_states.nc')
import numpy as np

def calculate_grid_cell_area(lat, resolution):
    # Earth’s radius in kilometers
    R = 6371.0
    # Convert latitude to radians
    lat_rad = np.deg2rad(lat)
    # Area of a grid cell in sq km (approximation)
    area = (np.pi / 180) * R**2 * np.abs(np.sin(lat_rad - resolution / 2) - np.sin(lat_rad + resolution / 2)) * resolution
    return area


In [26]:
area = pd.read_csv("local-data/1_openwater_area.csv")
polygons_with_geolocation = pd.read_csv("local-data/Hylakes_with_location.csv")

/tmp/ipykernel_8536/544922905.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  polygons_with_geolocation = pd.read_csv("local-data/Hylakes_with_location.csv")


In [95]:
area

,Hylak_id,1985-01-01,1985-02-01,1985-03-01,1985-04-01,1985-05-01,1985-06-01,1985-07-01,1985-08-01,1985-09-01,...,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01
0,2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.824253e+08,2.999365e+10,3.068883e+10,3.068883e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,1.365703e+09,3.069998e+10,3.071131e+10,3.071131e+10,3.071131e+10,3.071131e+10,1.706184e+10
1,3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.284760e+09,2.695556e+10,2.695554e+10,2.688677e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,1.109286e+10,2.698245e+10,2.698243e+10,2.691744e+10,2.691744e+10,2.691744e+10,2.093578e+10
2,4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.039779e+10,2.397008e+10,2.395869e+10,2.395673e+10,2.397002e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,2.080075e+10,2.399045e+10,2.398866e+10,2.400082e+10,2.400082e+10,1.413381e+10,0.000000e+00
3,5,8.209183e+10,7.479478e+10,8.209180e+09,1.460250e+10,7.301322e+10,8.213877e+10,8.213889e+10,8.213926e+10,8.213665e+10,...,0.000000e+00,0.000000e+00,3.285955e+10,8.214761e+10,8.214775e+10,8.214817e+10,8.214516e+10,8.214376e+10,8.214376e+10,8.214376e+10
4,6,5.789292e+10,5.789292e+10,3.023296e+10,4.374115e+10,5.789018e+10,5.788657e+10,5.789026e+10,5.788814e+10,5.787022e+10,...,8.361514e+09,3.087311e+10,5.788372e+10,5.787887e+10,5.788382e+10,5.788097e+10,5.785689e+10,5.784615e+10,5.784615e+10,5.784615e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427682,1427684,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1427683,1427685,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1427684,1427686,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1427685,1427687,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [96]:
def get_geolocation(location):
    geolocation = {}
    lake_area = location["Lake_area"]
    hylak = location["Hylak_id"]
    name = location["Lake_name"]
    country = location["Country"]
    latitude = location["Pour_lat"]
    longitude = location["Pour_long"]
    for row in range (location.shape[0]):
      
        geolocation[hylak[row]] = {"Name": name[row], "Country": country[row], "Latitude": latitude[row], "Longitude": longitude }

    return geolocation

In [104]:
def get_area(geo, area):
    area_per_id = {}
    hylak = area["Hylak_id"]
    for r in range(area.shape[0]):
        s = []
        for c in range(area.shape[1]):
            if geo[hylak[r]]["Latitude"] < 0:
                if "-01-01" in area.columns[c]:
                    s.append(area.iloc[r, c])
            else:
                if "-07-01" in area.columns[c]:
                    s.append(area.iloc[r,c])
        area_per_id[hylak[r]] = s
    return area_per_id

In [37]:
hylak = polygons_with_geolocation["Hylak_id"]
print(hylak[1])

2


In [32]:
polygons_with_geolocation

,OBJECTID,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,...,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,Shape_Length,Shape_Area
0,1,1,Caspian Sea,Russia,Europe,SWBD,1,0,377001.906250,15829.370117,...,200.500000,8110.642090,107883.000000,-29,-1.00,1.404108e+06,47.717708,45.591934,169.207580,40.900463
1,2,2,Great Bear,Canada,North America,CanVec,1,0,30450.640625,5331.720215,...,72.199997,535.187012,47577.699219,145,-1.00,1.476654e+05,-123.505546,65.138384,92.334674,6.055066
2,3,3,Great Slave,Canada,North America,CanVec,1,0,26734.289062,9814.160156,...,59.099998,4350.691895,4203.200195,148,-1.00,9.953123e+05,-117.617115,61.311727,151.105423,4.573550
3,4,4,Winnipeg,Canada,North America,CanVec,3,709,23923.039062,4018.320068,...,11.900000,2244.727051,1464.300049,215,-1.00,9.196115e+05,-97.863542,53.696359,46.691373,3.181086
4,5,5,Superior,United States of America,North America,SWBD,1,0,81843.921875,5248.359863,...,146.699997,2869.952881,48410.300781,179,-1.00,2.092195e+05,-84.460547,46.468593,60.032148,9.803349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427683,1427684,1427684,NaN,New Zealand,Oceania,SWBD,1,0,0.290000,2.300000,...,5.100000,0.125000,139.600006,1,4.83,4.400000e+00,169.147152,-52.592307,0.026054,0.000039
1427684,1427685,1427685,NaN,Australia,Oceania,SWBD,1,0,0.100000,1.660000,...,2.100000,-9999.000000,-9999.000000,1,1.04,-9.999000e+03,73.305108,-52.973622,0.019360,0.000013
1427685,1427686,1427686,NaN,Australia,Oceania,SWBD,1,0,0.240000,1.920000,...,3.700000,0.013000,792.400024,205,2.63,2.000000e-01,158.892278,-54.529970,0.022879,0.000033
1427686,1427687,1427687,NaN,Australia,Oceania,SWBD,1,0,0.340000,2.950000,...,6.500000,0.020000,1263.900024,163,7.58,4.000000e-01,158.889583,-54.594300,0.036477,0.000047


In [54]:
# 1 = 111 km
def urbanization(give_lat, give_lon):
    radius = 25
    #give_lat = 47.7231
    #give_lon = 86.9407
    urbanized_area_km2 = []
    radius_degrees = radius / 111  # Conversion from km to degrees
    
    start_year = 1985
    end_year = 2018
    
    for year in range(start_year, end_year + 1):
        data_for_year = data_urbanization.sel(time=slice(year, year))
        
        # Calculate latitude and longitude indices ranges
        lat_idx_range = np.where((data_for_year.latitude >= give_lat - radius_degrees) & 
                                 (data_for_year.latitude <= give_lat + radius_degrees))[0]
        lon_idx_range = np.where((data_for_year.longitude >= give_lon - radius_degrees) & 
                                 (data_for_year.longitude <= give_lon + radius_degrees))[0]
    
        # Process each point within the radius
        urban_counts = []
        for lat_idx in lat_idx_range:
            for lon_idx in lon_idx_range:
                LULC_state = data_for_year.LULC_states.isel(latitude=lat_idx, longitude=lon_idx).values
                # Check if LULC_state is in urban codes and sum
                urban_counts.append(np.sum(np.isin(LULC_state, [11, 22])))
    
        # Calculate urbanization percentage
        total_urban_count = np.sum(urban_counts)
        total_points = len(urban_counts)
        urbanization_percentage = (total_urban_count / total_points) * 100 if total_points > 0 else 0
    
        # Calculate urbanized area in km²
        total_area_km2 = math.pi * (radius ** 2)
        urbanized_area_km = urbanization_percentage * total_area_km2 / 100
        urbanized_area_km2.append(urbanized_area_km)
    return urbanized_area_km2
        #print(f"Year {year}: Urbanized area: {urbanized_area_km2} km², Urbanization: {urbanization_percentage}%")

In [ ]:
x_limits = [-1, 1]
y_limits = [0, 100]
geo = get_geolocation(polygons_with_geolocation)
hylak = polygons_with_geolocation["Hylak_id"]
id_area = get_area(geo, area)
for g in range(len(geo)):
    #import pdb; pdb.set_trace()
    urbanized_area_km2_array = urbanization(geo[hylak[g]]["Latitude"], geo[hylak[g]]["Longitude"][g])
    correlation, p_value = stats.pearsonr(urbanized_area_km2_array, id_area[hylak[g]])
    pcc_values.append(correlation)
#plt.figure()
plt.hist(values,bins=len(geo[hylak]), color = "black")
plt.xlim(x_limits)
plt.ylim(y_limits)
plt.xlabel("Correlation Coeficiants")
plt.ylabel("Frequency")
plt.title(f"Histagram Correlation Between Urbanization and Lake Surface Area")
plt.savefig(f"Histagram_of_Correlation_Between_Urbanization_and_Lake_Surface_Area_With_Ranges__{len(bins[hylak])}_Lakes.png")
#plt.close()

In [78]:
geo = get_geolocation(polygons_with_geolocation)

In [93]:
polygons_with_geolocation["Lake_area"]

0          377001.906250
1           30450.640625
2           26734.289062
3           23923.039062
4           81843.921875
               ...      
1427683         0.290000
1427684         0.100000
1427685         0.240000
1427686         0.340000
1427687         0.440000
Name: Lake_area, Length: 1427688, dtype: float64

In [48]:
print(len(geo))
print(len(data_for_year["LULC_states"]))

1427688
1


In [49]:
print(data_urbanization['LULC_states'])

<xarray.DataArray 'LULC_states' (time: 121, latitude: 18000, longitude: 36000)>
dask.array<open_dataset-LULC_states, shape=(121, 18000, 36000), dtype=float32, chunksize=(4, 667, 1334), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) float32 1.899e+03 1.9e+03 ... 2.018e+03 2.019e+03
  * latitude   (latitude) float32 90.0 89.99 89.97 89.96 ... -89.97 -89.99 -90.0
  * longitude  (longitude) float32 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Attributes:
    units:      numeric land use/cover codes
    long_name:  land use/cover category as numeric codes


In [47]:
data_urbanization

<xarray.Dataset>
Dimensions:           (time: 121, latitude: 18000, longitude: 36000)
Coordinates:
  * time              (time) float32 1.899e+03 1.9e+03 ... 2.018e+03 2.019e+03
  * latitude          (latitude) float32 90.0 89.99 89.97 ... -89.99 -90.0
  * longitude         (longitude) float32 -180.0 -180.0 -180.0 ... 180.0 180.0
Data variables:
    LULC_states       (time, latitude, longitude) float32 dask.array<chunksize=(4, 667, 1334), meta=np.ndarray>
    LULC_transitions  (time, latitude, longitude) float32 dask.array<chunksize=(4, 530, 1059), meta=np.ndarray>
Attributes:
    description:      HILDA+ land use/cover reconstruction - LULC states
    version:          GLOB-1.0-f
    info:             annual land cover/use maps 1899-2019
    temp_coverage:    1899-2019
    spat_coverage:    -180,-90: 180,90
    temp_resolution:  annual
    spat_resolution:  0.01 degree
    projection:       EPSG:4326 - WGS 84 – Geographic
    producer:         Winkler, Karina (WUR/KIT)